# CNN Transfer&Train&Predict

In [10]:
import requests
import json, os

url = "{0}:{1}".format(os.environ['HOSTNAME'] , "8000")

nn_id = "nn00004"
biz_cate = "ERP"
biz_sub_cate = "MRO"
nn_title = "MRO Image Classification"
nn_desc = "MRO Image Classification"
nn_wf_ver_info = "MRO Image Classification"
use_flag = "Y"
dirstr = "purpose?"
config = "N"
network_type = "cnn"
node_sub_menu = "data_image"

# get workflow version info
resp = requests.get('http://' + url + '/api/v1/type/common/target/nninfo/'+nn_id+'/version/')
data = json.loads(resp.json())

# get Active workflow version
wf_ver_id = 0
data = sorted(data, key=lambda k: k['pk'])
for config in data:
    if config["fields"]["active_flag"] == "Y":
        wf_ver_id = config["pk"]

wf_ver_id = str(wf_ver_id)
print("Network ID="+nn_id)
print("Active Version Workflow ID="+wf_ver_id)
print("")

node = "netconf_node"
resp = requests.put('http://' + url + '/api/v1/type/wf/state/netconf/detail/cnn/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',
                     json={
                         "config": {"learnrate": 0.001,
                                     "traincnt": 100,
                                     "batch_size":10000,
                                     "num_classes":10,
                                     "predictcnt": 10,
                                     "layeroutputs":32
                                     }
                         ,"layer1": {"type": "cnn",
                                     "active": "relu",
                                     "cnnfilter": [3, 3],
                                     "cnnstride": [1, 1],
                                     "maxpoolmatrix": [2, 2],
                                     "maxpoolstride": [2, 2],
                                     "padding": "SAME",
                                     "droprate": "0.8",
                                     "layercnt":2
                                    }
                         ,"layer2": {"type": "cnn",
                                     "active": "relu",
                                     "cnnfilter": [3, 3],
                                     "cnnstride": [1, 1],
                                     "maxpoolmatrix": [2, 2],
                                     "maxpoolstride": [2, 2],
                                     "padding": "SAME",
                                     "droprate": "0.8",
                                     "layercnt":1
                                    }
                          ,"out": {"active": "softmax",
                                   "node_out": 625,
                                   "padding": "SAME"
                                }
                        })
netconf = json.loads(resp.json())
print("Network Config : {0}".format(netconf))
print("")

node = "datasrc"
resp = requests.put('http://' + url + '/api/v1/type/wf/state/imgdata/src/local/form/file/prg/source/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',
                     json={
                            "preprocess": {"x_size": 32,
                                        "y_size": 32,
                                        "channel":3}
                         ,"labels":[]

                     })
dataconf = json.loads(resp.json())
print("Train Data Config : {0}".format(dataconf))
print("")

node = "eval_node"
resp = requests.put('http://' + url + '/api/v1/type/wf/state/netconf/detail/cnn/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',json={})
evalconf = json.loads(resp.json())
print("Eval Config : {0}".format(evalconf))
print("")

node = 'evaldata'
resp = requests.put('http://' + url + '/api/v1/type/wf/state/imgdata/src/local/form/file/prg/source/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',
                     json={
                            "preprocess": {"x_size": 32,
                                        "y_size": 32,
                                        "channel":3}
                         ,"labels":[]

                     })
edataconf = json.loads(resp.json())
print("Eval Data Config : {0}".format(edataconf))
print("")

Network ID=nn00004
Active Version Workflow ID=44

Network Config : {'modelname': 'model_nn00004_44', 'layer1': {'maxpoolmatrix': [2, 2], 'layercnt': 2, 'type': 'cnn', 'padding': 'SAME', 'cnnstride': [1, 1], 'maxpoolstride': [2, 2], 'active': 'relu', 'droprate': '0.8', 'cnnfilter': [3, 3]}, 'modelpath': '/hoya_model_root/nn00004/44/netconf_node', 'out': {'node_out': 625, 'active': 'softmax', 'padding': 'SAME'}, 'layer2': {'maxpoolmatrix': [2, 2], 'layercnt': 1, 'type': 'cnn', 'padding': 'SAME', 'cnnstride': [1, 1], 'maxpoolstride': [2, 2], 'active': 'relu', 'droprate': '0.8', 'cnnfilter': [3, 3]}, 'config': {'layeroutputs': 32, 'num_classes': 10, 'predictcnt': 10, 'learnrate': 0.001, 'batch_size': 10000, 'traincnt': 100}}

Train Data Config : {'preprocess': {'y_size': 32, 'x_size': 32, 'channel': 3}, 'source_path': '/hoya_src_root/nn00004/44/datasrc', 'labels': [], 'store_path': '/hoya_str_root/nn00004/44/datasrc'}

Eval Config : {'modelname': 'model_nn00004_44', 'modelpath': '/hoya_mod

# Trainning & Eval Data Transfer
(CNN Network Train을 할 수 있게 Data를 특정 장소에 전송해 준다.)<br>

In [11]:
# get workflow node list info (train, eval)
resp = requests.get('http://' + url + '/api/v1/type/wf/state/data/detail/upload/file/nnid/'+nn_id+'/ver/'+wf_ver_id+'/dir/'+node_sub_menu+'/')
datalist = json.loads(resp.json())

print("get file node list")
for node in datalist:
    print(datalist[node])
print("")

get file node list
{'nn_wf_node_name': 'evaldata', 'nn_wf_node_id': 'nn00004_44_evaldata', 'nn_wf_node_desc': 'Evaluation Data Node'}
{'nn_wf_node_name': 'datasrc', 'nn_wf_node_id': 'nn00004_44_datasrc', 'nn_wf_node_desc': 'Train Data Node'}



In [13]:
# Train Folder에 Data를 위치 시킬지 Eval Folder에 Data를 위치 시킬지 결정후 전송해야 한다.
print("create file node list")
for node in datalist:
    typepath = datalist[node]["nn_wf_node_name"]

    files = {'file1': open('/home/dev/hoyai/demo/data/cat_vs_dog.zip','rb'),
             'file2': open('/home/dev/hoyai/demo/data/sample_cnn_img.zip','rb')}

    resp = requests.post('http://' + url + '/api/v1/type/wf/state/data/detail/upload/file/nnid/'+nn_id+'/ver/'+wf_ver_id+'/dir/'+typepath+'/',
                         files=files)
    data = json.loads(resp.json())
    print(data)

create file node list
{'File1': {'File': 'File Create.(/hoya_src_root/nn00004/44/evaldata/cat_vs_dog.zip1)'}, 'File0': {'File': 'File Create.(/hoya_src_root/nn00004/44/evaldata/cat_vs_dog.zip1)'}}
{'File1': {'File': 'File Create.(/hoya_src_root/nn00004/44/datasrc/cat_vs_dog.zip1)'}, 'File0': {'File': 'File Create.(/hoya_src_root/nn00004/44/datasrc/cat_vs_dog.zip1)'}}


# CNN Network Training
(CNN Network Training을 실행한다.)<br> 

In [ ]:
resp = requests.post('http://' + url + '/api/v1/type/runmanager/state/train/nnid/'+nn_id+'/ver/'+wf_ver_id+'/')
data = json.loads(resp.json())

for train in data:
    if train != None and train != "" and train != {} and train != "status" and train != "result" and train["TrainResult"] != None:
        for tr in train["TrainResult"]:
            print(tr)

# CNN Network Predict
(CNN Network Predict를 실행한다.)<br>

In [15]:
files = {
         'files000001':  open('/home/dev/hoyai/demo/data/airplane/1air.jpg','rb')
        ,'files000002':  open('/home/dev/hoyai/demo/data/airplane/2air.jpg','rb')
        ,'files000003':  open('/home/dev/hoyai/demo/data/bolt/1bolt.jpg','rb')
        ,'files000004':  open('/home/dev/hoyai/demo/data/bolt/2bolt.jpg','rb')
        ,'files000005':  open('/home/dev/hoyai/demo/data/car/1car.jpg','rb')
        ,'files000006':  open('/home/dev/hoyai/demo/data/car/2car.jpg','rb')
        ,'files000007':  open('/home/dev/hoyai/demo/data/glove/1glove.jpg','rb')
        ,'files000008':  open('/home/dev/hoyai/demo/data/glove/2glove.jpg','rb')
        ,'files000009':  open('/home/dev/hoyai/demo/data/motor/1motor.jpg','rb')
        ,'files000010':  open('/home/dev/hoyai/demo/data/motor/2motor.jpg','rb')
        }
restURL = 'http://' + url + '/api/v1/type/service/state/predict/type/'+network_type+'/nnid/'+nn_id+'/ver/'+wf_ver_id+'/'

resp = requests.post(restURL,
                     files=files
                     )
data = json.loads(resp.json())

for train in data:
    print("FileName = "+train)
    print(data[train]['key'])
    print(data[train]['val'])
    print('')